# 5 .PROGRAM TO IMPLEMENT HOMOMORPHIC FILTERING



In [3]:
import logging
import numpy as np
#from google.colab.patches import cv2_imshow
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

class HomomorphicFilter:
    def __init__(self, a = 0.5, b = 1.5):
        self.a = float(a)
        self.b = float(b)

    def __butterworth_filter(self, I_shape, filter_params):
        P = I_shape[0]/2
        Q = I_shape[1]/2
        U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
        Duv = (((U-P)**2+(V-Q)**2)).astype(float)
        H = 1/(1+(Duv/filter_params[0]**2)**filter_params[1])
        return (1 - H)

    def __apply_filter(self, I, H):
        H = np.fft.fftshift(H)
        I_filtered = (self.a + self.b*H)*I
        return I_filtered

    def filter(self, I, filter_params, filter='butterworth', H = None):
        if len(I.shape) is not 2:
            raise Exception('Improper image')
        I_log = np.log1p(np.array(I, dtype="float"))
        I_fft = np.fft.fft2(I_log)
        if filter=='butterworth':
            H = self.__butterworth_filter(I_shape = I_fft.shape, filter_params = filter_params)
        elif filter=='external':
            print('external')
            if len(H.shape) is not 2:
                raise Exception('Invalid external filter')
        else:
            raise Exception('Selected filter not implemented')
        
        I_fft_filt = self.__apply_filter(I = I_fft, H = H)
        I_filt = np.fft.ifft2(I_fft_filt)
        I = np.exp(np.real(I_filt))-1
        return np.uint8(I)
def main():
    import cv2
    img_path = 'virat-kohli-indian-cricketer-team-india-hd-wallpaper-preview.jpg'
    img_path_in =  img_path
    img=cv2.imread("virat-kohli-indian-cricketer-team-india-hd-wallpaper-preview.jpg")
    img = cv2.imread(img_path_in)[:, :, 0]
    homo_filter = HomomorphicFilter(a = 0.85, b = 0.75)
    img_filtered = homo_filter.filter(I=img, filter_params=[30,2])
    result=np.hstack((img,img_filtered))
    cv2.imshow("Homomorphic Filtering",result)

In [4]:
main()